# DSC 40B: Merge Sort

### By: Daniel Lee, Udaikaran Singh, and Justin Eldridge

This is the first of several "demo" notebooks that will be released throughout the quarter. These notebooks are supplementary -- the only things you will "need" to know for the exams will be covered in lecture, homework, discussion, and the course notes. But any "Exercises" which appear below can be thought of as practice problems, and it is recommended that you try them.

In [ ]:
# miscellaneous imports
import numpy as np
import time
import matplotlib.pyplot as plt
from SortHelper import CallOrder

plt.style.use('ggplot')

___

## Merge Sort

Below is the code for merge sort as it appears in the course notes (and how it appeared in lecture).
___

In [ ]:
import math

def mergesort(arr):
    """Sorts `arr` by modifying it rather than returning new list.
    
    `arr` should be a Python list.
    """
    if len(arr) > 1: 
        # split the array into halves
        mid = math.floor(len(arr) / 2)
        left = arr[:mid]
        right = arr[mid:]
        
        # sort the left half  
        mergesort(left)
        # sort the right half
        mergesort(right)
        
        # merge the sorted halves, storing result in arr
        merge(left, right, arr)
        
def merge(left, right, arr):
    """Merge two sorted lists, left and right, into arr."""
    left.append(float('inf'))
    right.append(float('inf'))
        
    for ix in range(len(arr)):
        if left[0] < right[0]:
            arr[ix] = left.pop(0)
        else:
            arr[ix] = right.pop(0)

___

## Demo: Runtime Analysis

___

One of the main advantages of merge sort is that it is fast: it has time complexity $\Theta(n \log n)$, where $n$ is the size of the input.

We derived this result in class by solving a recurrence, but let's try to verify it empirically. To do so, we will run `mergesort` on inputs of varying size, time how long each call takes, and plot the results.

First, we make a list of all of the input sizes that will be tried. The below makes an array in which each entry is twice the previous.

In [ ]:
sizes = 2 ** np.arange(1, 15)
sizes

Now we loop through `sizes`, creating a random array of each size and timing `mergesort` when it is called on that array. We store the times in `mergesort_times`.

In [ ]:
mergesort_times = []
for size in sizes:
    lst = list(np.random.randint(100, size=size))
    
    start_time = time.time()
    mergesort(lst)
    end_time = time.time()
    
    duration = end_time - start_time
    mergesort_times.append(duration)

Let's plot `mergesort_times` against `sizes`:

In [ ]:
plt.plot(sizes, mergesort_times, marker='x')
plt.xlabel('Size')
plt.ylabel('Time (s)')

If you re-run the above cell over and over again, you'll notice that the plot will change a little bit every time. This is because the time it takes to run `mergesort` on any given input will fluctuate depending on what your computer is doing in the background. Interestingly, most times there will be a little "hump" around $n = 2500$: why do you think that is?

Now, we know that merge sort has $\Theta(n \log n)$ time complexity, so the above function should look like $n \log n$ when $n$ is large. Does it? It's hard to see... Let's try to fit the best line of the form $a n \log n + b$ to our timings. We can do so using `np.polyfit`

In [ ]:
a, b = np.polyfit(sizes * np.log(sizes), mergesort_times, deg=1)
plt.plot(
    sizes, 
    a * sizes * np.log(sizes) + b, 
    color='C1', 
    linestyle='--',
    label='Best Fit'
)
plt.plot(sizes, mergesort_times, label='Empirical')

plt.title('Fit of $a n \log n + b$')
plt.legend()

It looks like a good fit. But let's try to fit a quadratic function, $an^2 + bn + c$, just to see what we get:

In [ ]:
a, b, c = np.polyfit(sizes, mergesort_times, deg=2)
plt.plot(
    sizes, 
    a * sizes**2 + b * sizes + c, 
    color='C1', 
    linestyle='--',
    label='Best Fit'
)
plt.plot(sizes, mergesort_times, label='Empirical')

plt.title('Fit of $a n^2 + b n + c$')
plt.legend()

It is probably an even better fit (if it isn't, you might have been unlucky with your timings -- run the last few cells again)! So does merge sort actually take quadratic time?

No! Take a look at the coefficients of the quadratic fit:

In [ ]:
a

In [ ]:
b

In [ ]:
c

$a$ should be very close to zero. Since $a$ is the coefficient on $n^2$, this tells us that the rate of growth of the timings is mostly linear.

But even saying that is misleading. What this exercise has really demonstrated is one of the pitfalls of assessing efficiency with empirical timings instead of analyzing the code of the algorithm itself. Empirical timings will have noise, and it turns out to be really difficult to distinguish a noisy function that grows like $\Theta(n \log n)$ from a noisy function that grows like $\Theta(n)$.

Now let's try to time selection sort. Below is the same code we saw in the course notes and in lecture:

In [ ]:
def selection_sort(arr):
    """In-place selection sort."""
    n = len(arr)
    if n <= 1:
        return
    for barrier_ix in range(n-1):
        min_ix = find_minimum(arr, start=barrier_ix)
        arr[barrier_ix], arr[min_ix] = arr[min_ix], arr[barrier_ix] # swap
        
def find_minimum(arr, start):
    """Finds index of minimum. Assumes arr is non-empty."""
    n = len(arr)
    min_value = arr[start]
    min_ix = start
    for i in range(start + 1, n):
        if arr[i] < min_value:
            min_value = arr[i]
            min_ix = i
    return min_ix

We'll use the same `sizes` as above, and time `selection_sort` on each. The below code might take a second or two to run, because `selection_sort` is asymptotically slower than `mergesort`:

In [ ]:
selection_sort_times = []

for size in sizes:
    lst = list(np.random.randint(100, size=size))
    
    start_time = time.time()
    selection_sort(lst)
    end_time = time.time()
    
    duration = end_time - start_time
    selection_sort_times.append(duration)

In [ ]:
plt.plot(sizes, selection_sort_times, marker='x')
plt.xlabel('Size')
plt.ylabel('Time (s)')

Remember that selection sort takes $\Theta(n^2)$ time, and the plot above seems to grow quadratically. But we can't be sure from the plot alone that the time complexity isn't, say, $n^2 \log n$ or $n^{1.9}$, since those functions would have very similar plots.

Lastly, let's plot the timings of merge sort and selection sort together to compare them:

In [ ]:
plt.plot(sizes, selection_sort_times, label='Selection Sort', marker='x')
plt.plot(sizes, mergesort_times, label='Merge Sort', marker='x')
plt.xlabel('Size')
plt.ylabel('Time (s)')
plt.legend()

Merge sort looks almost flat -- that is because $n \log n$ grows so much slower than $n^2$. What if we "zoom in" to the timings for arrays of smaller size, where the algorithms are comparable?

In [ ]:
# zoom into the first k timings
k = 6

plt.plot(sizes[:k], selection_sort_times[:k], label='Selection Sort', marker='x')
plt.plot(sizes[:k], mergesort_times[:k], label='Merge Sort', marker='x')
plt.xlabel('Size')
plt.ylabel('Time (s)')
plt.legend()

What you should see is that selection sort is actually faster than merge sort when the array is small! Remember that asymptotic time complexity is just that -- asymptotic. Merge sort will be the faster algorithm if the input is large enough. But apparently for small enough inputs, merge sort has enough overhead that selection sort wins. If you change $k$ in the cell above by making it larger, you'll eventually find the "crossing point" where `mergesort` is the faster function.

In fact, "real life" implementations of merge sort take advantage of this observation. Once the input becomes small enough, merge sort doesn't make a recursive call, and instead uses another sorting algorithm (usually *insertion sort*) to sort each half. The time complexity doesn't change, but the algorithm is faster by a constant factor.

___

# Exercise: Call Order

___

## Animation
![animation2](https://i.imgur.com/HU2tfzo.gif)


Animations like these are often used to show how merge sort works, but they also tend to hide what is happening inside the computer, and when. The animation might lead you to believe that the left and right halves of the array are sorted simultaneously, but this isn't the case. In fact, the left half of the array is sorted before we even start to sort the right half!

As an exercise to demonstrate this, look at the code for `mergesort`, especially where the recursive calls are. Now try to guess the order of the recursive calls made when sorting the array: [1,3,2,6,5,1,7,2]. Run the cell below to check your work.

Hint: the argument of the first call is [1, 3, 2, 6, 5, 1, 7, 2]. The argument of the second call is [1, 3, 2, 6].

In [ ]:
x = [1,3,2,6,5,1,7,2]
CallOrder(x)

Now replace the ... below with your own favorite list of numbers. Before running the code, attempt to guess the call order. Try a list of length 5 and a list of length 9 to make sure you have the right idea. 

In [ ]:
CallOrder(...)

___

## Exercise: How Many Times Printed?

___

In this exercise, you should:

 1. Remove a comment on one of the three `print` statements
 2. Guess how many times that statement will be printed when `arr` has 7 elements.
 3. Run the code below to check your answer

You should try all 3 possible positions.


Hint: A question like this may show up on future exams.

In [ ]:
import math

def noisy_mergesort(arr):
    """Sorts `arr` by modifying it rather than returning new list.
    
    `arr` should be a Python list.
    """
    print('Position 1')
    if len(arr) > 1: 
        # split the array into halves
        mid = math.floor(len(arr) / 2)
        left = arr[:mid]
        right = arr[mid:]
        
        # print('Position 2')
        # sort the left half  
        noisy_mergesort(left)
        # sort the right half
        noisy_mergesort(right)
        
        # merge the sorted halves, storing result in arr
        noisy_merge(left, right, arr)
        
def noisy_merge(left, right, arr):
        left.append(float('inf'))
        right.append(float('inf'))
        
        for ix in range(len(arr)):
            # print('Position 3')
            if left[0] < right[0]:
                arr[ix] = left.pop(0)
            else:
                arr[ix] = right.pop(0)

In [ ]:
noisy_mergesort([1,3,2,6,5,1,7])

Now try to come up with a general formula that predicts how many times "Position 1" is printed when run on an input of size $n$ (you can assume $n$ is a power of 2 to make things nice). Do the same for "Position 2" and "Position 3". Check your work by running `noisy_mergesort` with the corresponding lines uncommented and with inputs of various sizes, and verify that your formula made the right predictions.